**텐서플로에서 데이터 적재와 전처리하기**

# 설정

먼저 몇 개의 모듈을 임포트한다. 맷플롯립 그림을 저장하는 함수를 준비한다.

In [1]:
# 공통 모듈 임포트
import matplotlib as mpl
import os
import matplotlib.pyplot as plt

# 깔끔한 그래프 출력을 위해
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
# 그림을 저장할 위치
PROJECT_ROOT_DIR = '.'
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, 'images')
os.makedirs(IMAGES_PATH, exist_ok=True)


def save_fig(fig_id, tight_layout=True, fig_extension='png', resolution=300):
    path = os.path.join(IMAGES_PATH, f'{fig_id}.{fig_extension}')
    print(f'그림 저장: {fig_id}')
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## 데이터셋

In [2]:
import tensorflow as tf

X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
dataset = tf.data.Dataset.range(10)

In [4]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


In [5]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


In [6]:
dataset = dataset.map(lambda x: x * 2)

In [7]:
for item in dataset:
    print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int64)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int64)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int64)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int64)
tf.Tensor([16 18], shape=(2,), dtype=int64)


In [8]:
dataset = dataset.unbatch()

In [9]:
dataset = dataset.filter(lambda x: x < 10)  # < 10 항목만 유지

In [10]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)


In [11]:
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([2 1 4 0 3 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 5 0 2 1 3 4], shape=(7,), dtype=int64)
tf.Tensor([9 5 7 6 8 0 9], shape=(7,), dtype=int64)
tf.Tensor([3 1 4 6 7 8 5], shape=(7,), dtype=int64)
tf.Tensor([9 2], shape=(2,), dtype=int64)


## 캘리포니아 주택 데이터셋을 여러 개의 CSV로 나누기

캘리포니아 주택 데이터셋을 로드하고 준비한다. 먼저 로드한 다음 훈련 세트, 검증 세트, 테스트 세트로 나눈다. 마지막으로 스케일을 조정한다:

In [12]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target.reshape(-1, 1))
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)
scaler = StandardScaler()
scaler.fit(X_train)
X_mean = scaler.mean_
X_std = scaler.scale_

메모리에 맞지 않는 매우 큰 데이터셋인 경우 일반적으로 먼저 여러 개의 파일로 나누고 텐서플로에서 이 파일들을 병렬로 읽게한다. 데모를 위해 주택 데이터셋을 20개의 CSV 파일로 나누어 본다:

In [13]:
def save_to_multiple_csv_files(data, name_prefix, header=None, n_parts=10):
    housing_dir = os.path.join('datasets', 'housing')
    os.makedirs(housing_dir, exist_ok=True)
    path_format = os.path.join(housing_dir, 'my_{}_{:02d}.csv')
    filepaths = []
    m = len(data)
    for file_idx, row_indices in enumerate(np.array_split(np.arange(m), n_parts)):
        part_csv = path_format.format(name_prefix, file_idx)
        filepaths.append(part_csv)
        with open(part_csv, 'wt', encoding='utf-8') as f:
            if header is not None:
                f.write(header)
                f.write('\n')
            for row_idx in row_indices:
                f.write(','.join([repr(col) for col in data[row_idx]]))
                f.write('\n')
    return filepaths

In [14]:
import numpy as np

train_data = np.c_[X_train, y_train]
valid_data = np.c_[X_valid, y_valid]
test_data = np.c_[X_test, y_test]
header_cols = housing.feature_names + ['MedianHouseValue']
header = ','.join(header_cols)
train_filepaths = save_to_multiple_csv_files(train_data, 'train', header, n_parts=20)
valid_filepaths = save_to_multiple_csv_files(valid_data, 'valid', header, n_parts=10)
test_filepaths = save_to_multiple_csv_files(test_data, 'test', header, n_parts=10)

좋다. 이 CSV 파일 중 하나에서 몇 줄을 출력해 본다:

In [15]:
import pandas as pd

pd.read_csv(train_filepaths[0]).head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
0,2.8750,16.0,4.808344,1.065189,1969.0,2.567145,32.82,-116.77,2.229
1,3.3333,40.0,4.668498,1.082418,1156.0,2.117216,34.16,-118.24,3.743
2,2.7540,21.0,2.642541,1.002954,2300.0,3.397341,34.20,-118.59,1.798
3,9.7066,38.0,8.750000,1.100000,129.0,6.450000,34.07,-117.92,1.825
4,5.3185,52.0,6.210024,0.997613,1137.0,2.713604,34.18,-118.28,3.580


텍스트 파일로 읽으면 다음과 같다:

In [16]:
with open(train_filepaths[0]) as f:
    for _ in range(5):
        print(f.readline(), end='')

MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedianHouseValue
2.875,16.0,4.808344198174707,1.0651890482398958,1969.0,2.5671447196870925,32.82,-116.77,2.229
3.3333,40.0,4.668498168498169,1.0824175824175823,1156.0,2.1172161172161172,34.16,-118.24,3.743
2.754,21.0,2.642540620384047,1.0029542097488922,2300.0,3.397341211225997,34.2,-118.59,1.798
9.7066,38.0,8.75,1.1,129.0,6.45,34.07,-117.92,1.825


In [17]:
train_filepaths

['datasets\\housing\\my_train_00.csv',
 'datasets\\housing\\my_train_01.csv',
 'datasets\\housing\\my_train_02.csv',
 'datasets\\housing\\my_train_03.csv',
 'datasets\\housing\\my_train_04.csv',
 'datasets\\housing\\my_train_05.csv',
 'datasets\\housing\\my_train_06.csv',
 'datasets\\housing\\my_train_07.csv',
 'datasets\\housing\\my_train_08.csv',
 'datasets\\housing\\my_train_09.csv',
 'datasets\\housing\\my_train_10.csv',
 'datasets\\housing\\my_train_11.csv',
 'datasets\\housing\\my_train_12.csv',
 'datasets\\housing\\my_train_13.csv',
 'datasets\\housing\\my_train_14.csv',
 'datasets\\housing\\my_train_15.csv',
 'datasets\\housing\\my_train_16.csv',
 'datasets\\housing\\my_train_17.csv',
 'datasets\\housing\\my_train_18.csv',
 'datasets\\housing\\my_train_19.csv']

## 입력 파이프라인 만들기

In [18]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths)

In [19]:
for filepath in filepath_dataset:
    print(filepath)

tf.Tensor(b'datasets\\housing\\my_train_02.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_17.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_06.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_03.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_04.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_16.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_09.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_15.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_05.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_14.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_01.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_10.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_07.csv', shape=(), dtype=string)
tf.Tensor(b'datasets\\housing\\my_train_13.csv', sh

In [20]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1), cycle_length=n_readers
)

In [21]:
for line in dataset.take(5):
    print(line.numpy())

b'2.4375,52.0,5.019607843137255,0.9859943977591037,1049.0,2.9383753501400562,36.76,-119.81,0.571'
b'2.675,37.0,3.404833836858006,0.9879154078549849,492.0,1.486404833836858,33.77,-118.17,2.417'
b'5.1314,15.0,29.074766355140188,6.08411214953271,276.0,2.5794392523364484,37.06,-119.32,1.792'
b'2.2656,23.0,6.690789473684211,1.4342105263157894,387.0,2.5460526315789473,39.9,-120.74,0.882'
b'2.7321,28.0,5.043269230769231,1.1298076923076923,705.0,3.389423076923077,36.85,-121.79,1.5'


네 번째 필드의 4는 문자열로 해석된다:

In [22]:
record_defaults = [0, np.nan, tf.constant(np.nan, dtype=tf.float64), 'Hello', tf.constant([])]
parsed_fields = tf.io.decode_csv('1,2,3,4,5', record_defaults)
parsed_fields

[<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.0>,
 <tf.Tensor: shape=(), dtype=float64, numpy=3.0>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'4'>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]

누락된 값은 제공된 기본값으로 대체된다:

In [23]:
parsed_fields = tf.io.decode_csv(',,,,5', record_defaults)
parsed_fields

[<tf.Tensor: shape=(), dtype=int32, numpy=0>,
 <tf.Tensor: shape=(), dtype=float32, numpy=nan>,
 <tf.Tensor: shape=(), dtype=float64, numpy=nan>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'Hello'>,
 <tf.Tensor: shape=(), dtype=float32, numpy=5.0>]

다섯 번째 필드는 필수이다(기본값을 `tf.constant([])`로 지정했기 때문에). 따라서 값을 전달하지 않으면 예외가 발생한다:

In [24]:
try:
    parsed_fields = tf.io.decode_csv(',,,,', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

{{function_node __wrapped__DecodeCSV_OUT_TYPE_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Field 4 is required but missing in record 0! [Op:DecodeCSV]


필드 개수는 `record_defaults`에 있는 필드 개수와 정확히 맞아야 한다:

In [25]:
try:
    parsed_fields = tf.io.decode_csv('1,2,3,4,5,6,7', record_defaults)
except tf.errors.InvalidArgumentError as ex:
    print(ex)

{{function_node __wrapped__DecodeCSV_OUT_TYPE_5_device_/job:localhost/replica:0/task:0/device:CPU:0}} Expect 5 fields but have 7 in record 0 [Op:DecodeCSV]


In [26]:
n_inputs = 8  # X_train.shape[-1]


@tf.function
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

In [27]:
preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([ 0.16774078,  1.1986382 , -0.0455903 , -0.52083874, -0.5073813 ,
        -0.46517077,  0.86297166, -1.2996906 ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.782], dtype=float32)>)

In [28]:
def csv_reader_dataset(
        filepaths,
        repeat=1,
        n_readers=5,
        n_read_threads=None,
        shuffle_buffer_size=10000,
        n_parse_threads=5,
        batch_size=32
):
    dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers,
        num_parallel_calls=n_read_threads
    )
    dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.batch(batch_size)
    return dataset.prefetch(1)

In [29]:
train_set = csv_reader_dataset(train_filepaths, batch_size=3)
for X_batch, y_batch in train_set.take(2):
    print('X =', X_batch)
    print('y =', y_batch)
    print()

X = tf.Tensor(
[[ 1.0548339  -0.6253646   0.50441206 -0.25903296  0.489802    0.12015844
  -0.63585055  0.42335564]
 [ 0.2619429   1.833074   -0.14657022 -0.15796822 -0.35886467 -0.26189885
   0.7877957  -1.1453148 ]
 [ 1.7785524  -0.7839736   0.8398351  -0.42026544  0.62771034  0.4990901
  -0.89426756  0.80680656]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[2.21 ]
 [2.236]
 [3.018]], shape=(3, 1), dtype=float32)

X = tf.Tensor(
[[ 0.7675666  -0.3874512   0.6496367  -0.27316982  0.15121937  0.03598217
  -1.3171314   1.2052004 ]
 [-0.6668316  -1.4977138  -0.15116398 -0.20659451  0.39255896 -0.269876
   0.3367385  -0.01487831]
 [-0.46496966  0.16768008 -0.06912699 -0.12527053 -0.5533508  -0.25151363
   1.9389265  -1.0257982 ]], shape=(3, 8), dtype=float32)
y = tf.Tensor(
[[2.125]
 [0.803]
 [0.967]], shape=(3, 1), dtype=float32)



In [30]:
train_set = csv_reader_dataset(train_filepaths, repeat=None)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

In [31]:
from tensorflow import keras

keras.backend.clear_session()
model = keras.models.Sequential(
    [keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]), keras.layers.Dense(1)]
)

In [32]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(learning_rate=1e-3))

In [33]:
batch_size = 32
model.fit(train_set, steps_per_epoch=len(X_train) // batch_size, epochs=10, validation_data=valid_set)

Epoch 1/10
362/362 [==============================] - 3s 7ms/step - loss: 2.1184 - val_loss: 1.5635
Epoch 2/10
362/362 [==============================] - 1s 3ms/step - loss: 0.8452 - val_loss: 0.9477
Epoch 3/10
362/362 [==============================] - 1s 3ms/step - loss: 0.6319 - val_loss: 0.7448
Epoch 4/10
362/362 [==============================] - 1s 3ms/step - loss: 0.6016 - val_loss: 0.6533
Epoch 5/10
362/362 [==============================] - 1s 3ms/step - loss: 0.5436 - val_loss: 0.6126
Epoch 6/10
362/362 [==============================] - 1s 3ms/step - loss: 0.5294 - val_loss: 0.6034
Epoch 7/10
362/362 [==============================] - 1s 3ms/step - loss: 0.5078 - val_loss: 0.5449
Epoch 8/10
362/362 [==============================] - 1s 3ms/step - loss: 0.5023 - val_loss: 0.5304
Epoch 9/10
362/362 [==============================] - 1s 3ms/step - loss: 0.4836 - val_loss: 0.5206
Epoch 10/10
362/362 [==============================] - 1s 3ms/step - loss: 0.4627 - val_loss: 0.5456

In [34]:
model.evaluate(test_set, steps=len(X_test) // batch_size)

161/161 [==============================] - 0s 2ms/step - loss: 0.9282


0.9282187819480896

In [35]:
new_set = test_set.map(lambda X, y: X)  # 대신 test_set을 전달할 수 있다. Keras는 레이블을 무시한다.
X_new = X_test
model.predict(new_set, steps=len(X_new) // batch_size)

161/161 [==============================] - 0s 1ms/step


array([[1.0731418],
       [2.053124 ],
       [3.3935633],
       ...,
       [1.8454084],
       [2.3096504],
       [3.0473442]], dtype=float32)

In [36]:
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error
n_epochs = 5
batch_size = 32
n_steps_per_epoch = len(X_train) // batch_size
total_steps = n_epochs * n_steps_per_epoch
global_step = 0
for X_batch, y_batch in train_set.take(total_steps):
    global_step += 1
    print(f'\rGlobal step {global_step}/{total_steps}', end='')
    with tf.GradientTape() as tape:
        y_pred = model(X_batch)
        main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
        loss = tf.add_n([main_loss] + model.losses)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

Global step 1810/1810

In [37]:
keras.backend.clear_session()

In [38]:
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error


@tf.function
def train(model, n_epochs, batch_size=32, n_readers=5, n_read_threads=5, shuffle_buffer_size=10000, n_parse_threads=5):
    train_set = csv_reader_dataset(
        train_filepaths,
        repeat=n_epochs,
        n_readers=n_readers,
        n_read_threads=n_read_threads,
        shuffle_buffer_size=shuffle_buffer_size,
        n_parse_threads=n_parse_threads,
        batch_size=batch_size
    )
    for X_batch, y_batch in train_set:
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))


train(model, 5)

In [39]:
optimizer = keras.optimizers.Nadam(learning_rate=0.01)
loss_fn = keras.losses.mean_squared_error


@tf.function
def train(model, n_epochs, batch_size=32, n_readers=5, n_read_threads=5, shuffle_buffer_size=10000, n_parse_threads=5):
    train_set = csv_reader_dataset(
        train_filepaths,
        repeat=n_epochs,
        n_readers=n_readers,
        n_read_threads=n_read_threads,
        shuffle_buffer_size=shuffle_buffer_size,
        n_parse_threads=n_parse_threads,
        batch_size=batch_size
    )
    n_steps_per_epoch = len(X_train) // batch_size
    total_steps = n_epochs * n_steps_per_epoch
    global_step = 0
    for X_batch, y_batch in train_set.take(total_steps):
        global_step += 1
        if tf.equal(global_step % 100, 0):
            tf.print("\rGlobal step", global_step, "/", total_steps)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))


train(model, 5)

Global step 100 / 1810
Global step 200 / 1810
Global step 300 / 1810
Global step 400 / 1810
Global step 500 / 1810
Global step 600 / 1810
Global step 700 / 1810
Global step 800 / 1810
Global step 900 / 1810
Global step 1000 / 1810
Global step 1100 / 1810
Global step 1200 / 1810
Global step 1300 / 1810
Global step 1400 / 1810
Global step 1500 / 1810
Global step 1600 / 1810
Global step 1700 / 1810
Global step 1800 / 1810


`Dataset` 클래스에 있는 메서드의 간략한 설명이다:

In [40]:
for m in dir(tf.data.Dataset):
    if not (m.startswith('_') or m.endswith('_')):
        func = getattr(tf.data.Dataset, m)
        if hasattr(func, '__doc__'):
            print('● {:21s}{}'.format(m + '()', func.__doc__.split('\n')[0]))

● apply()              Applies a transformation function to this dataset.
● as_numpy_iterator()  Returns an iterator which converts all elements of the dataset to numpy.
● batch()              Combines consecutive elements of this dataset into batches.
● bucket_by_sequence_length()A transformation that buckets elements in a `Dataset` by length.
● cache()              Caches the elements in this dataset.
● cardinality()        Returns the cardinality of the dataset, if known.
● choose_from_datasets()Creates a dataset that deterministically chooses elements from `datasets`.
● concatenate()        Creates a `Dataset` by concatenating the given dataset with this dataset.
● element_spec()       The type specification of an element of this dataset.
● enumerate()          Enumerates the elements of this dataset.
● filter()             Filters this dataset according to `predicate`.
● flat_map()           Maps `map_func` across this dataset and flattens the result.
● from_generator()     Create

## `TFRecord` 이진 포맷

TFRecord 파일은 단순히 이진 레코드의 리스트이다. `tf.io.TFRecordWriter`를 사용해 만들 수 있다:

In [41]:
with tf.io.TFRecordWriter('my_data.tfrecord') as f:
    f.write(b'This is the first record')
    f.write(b'And this is the second record')

그리고 `tf.data.TFRecordDataset` 사용해 읽을 수 있다.:

In [42]:
filepaths = ['my_data.tfrecord']
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


하나의 `TFRecordDataset`로 여러 개의 TFRecord 파일을 읽을 수 있다. 기본적으로 한 번에 하나의 파일만 읽지만 `num_parallel_reads=3`와 같이 지정하면 동시에 3개를 읽고 레코드를 번갈아 반환한다:

In [43]:
filepaths = [f'my_test_{i}.tfrecord' for i in range(5)]
for i, filepath in enumerate(filepaths):
    with tf.io.TFRecordWriter(filepath) as f:
        for j in range(3):
            f.write(f'File {i} record {j}'.encode('utf-8'))
dataset = tf.data.TFRecordDataset(filepaths, num_parallel_reads=3)
for item in dataset:
    print(item)

tf.Tensor(b'File 0 record 0', shape=(), dtype=string)
tf.Tensor(b'File 1 record 0', shape=(), dtype=string)
tf.Tensor(b'File 2 record 0', shape=(), dtype=string)
tf.Tensor(b'File 0 record 1', shape=(), dtype=string)
tf.Tensor(b'File 1 record 1', shape=(), dtype=string)
tf.Tensor(b'File 2 record 1', shape=(), dtype=string)
tf.Tensor(b'File 0 record 2', shape=(), dtype=string)
tf.Tensor(b'File 1 record 2', shape=(), dtype=string)
tf.Tensor(b'File 2 record 2', shape=(), dtype=string)
tf.Tensor(b'File 3 record 0', shape=(), dtype=string)
tf.Tensor(b'File 4 record 0', shape=(), dtype=string)
tf.Tensor(b'File 3 record 1', shape=(), dtype=string)
tf.Tensor(b'File 4 record 1', shape=(), dtype=string)
tf.Tensor(b'File 3 record 2', shape=(), dtype=string)
tf.Tensor(b'File 4 record 2', shape=(), dtype=string)


In [44]:
options = tf.io.TFRecordOptions(compression_type='GZIP')
with tf.io.TFRecordWriter('my_compressed.tfrecord', options) as f:
    f.write(b'This is the first record')
    f.write(b'And this is the second record')

In [45]:
dataset = tf.data.TFRecordDataset(['my_compressed.tfrecord'], compression_type='GZIP')
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


### 프로토콜 버퍼 개요

이 절을 위해서는 [프로토콜 버퍼를 설치](https://developers.google.com/protocol-buffers/docs/downloads)해야 한다. 일반적으로 텐서플로를 사용할 때 프로토콜 버퍼를 설치할 필요는 없다. 텐서플로는 `tf.train.Example` 타입의 프로토콜 버퍼를 만들고 파싱할 수 있는 함수를 제공하며 보통의 경우 충분하다. 하지만 이 절에서는 자체적인 프로토콜 버퍼를 간단히 만들어 보겠다. 따라서 프로토콜 버퍼 컴파일러(`protoc`)가 필요하다. 이를 사용해 프로토콜 버퍼 정의를 컴파일하여 코드에서 사용할 수 있는 파이썬 모듈을 만들겠다.

먼저 간단한 프로토콜 버퍼 정의를 작성해 본다:

In [65]:
%%writefile person.proto
syntax = "proto3";
message Person {
  string name = 1;
  int32 id = 2;
  repeated string email = 3;
}

Overwriting person.proto


이 정의를 컴파일한다(`--descriptor_set_out`와 `--include_imports` 옵션은 아래 `tf.io.decode_proto()` 예제를 위해서 필요하다):

In [66]:
!protoc person.proto --python_out=. --descriptor_set_out=person.desc --include_imports

In [67]:
%ls person*

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: C460-F1D1

 C:\Users\kyun\Desktop\loading_and_preprocessing_data 디렉터리

2022-11-05  오후 09:40                92 person.desc
2022-11-05  오후 09:40               108 person.proto
2022-11-05  오후 09:40               986 person_pb2.py
               3개 파일               1,186 바이트
               0개 디렉터리  113,741,611,008 바이트 남음


In [68]:
from person_pb2 import Person

person = Person(name='Al', id=123, email=['a@b.com'])  # Person 생성
print(person)  # Person 출력

name: "Al"
id: 123
email: "a@b.com"



In [70]:
person.name  # 필드 읽기

'Al'

In [71]:
person.name = 'Alice'  # 필드 수정

In [72]:
person.email[0]  # 배열처럼 사용할 수 있는 반복 필드

'a@b.com'

In [73]:
person.email.append('c@d.com')  # 이메일 추가

In [74]:
s = person.SerializeToString()  # 바이트 문자열로 직렬화
s

b'\n\x05Alice\x10{\x1a\x07a@b.com\x1a\x07c@d.com'

In [75]:
person2 = Person()  # 새로운 Person 생성
person2.ParseFromString(s)  # 바이트 문자열 파싱 (27 바이트)

27

In [76]:
person == person2  # 동일

True